In [1]:
import pandas as pd
import numpy as np
import os
import requests

In [31]:
%run Utilities.ipynb
%run GetAPIKey.ipynb

In [5]:
wd = 'C:\\Users\\DTRManning\\Desktop\\IndependentProjects\\BatteryOutageSizing'

In [6]:
### Plan
# Get household consumption profile
# Get rooftop PV profile (PVWatts?) - https://developer.nrel.gov/docs/solar/
# Dispatch battery to self consume 100% of household usage (charge off solar, discharge to serve load)

In [25]:
### Household profile data ###
# Reference: https://data.openei.org/submissions/153
# Data location: https://openei.org/datasets/files/961/pub/

# Construct API call
ResLoadAPI = {}
ResLoadAPI['baseURL']      = 'https://openei.org/datasets/files/961/pub'
ResLoadAPI['dataCategory'] = 'RESIDENTIAL_LOAD_DATA_E_PLUS_OUTPUT'
# Options for case are LOW, BASE, and HIGH
ResLoadAPI['case']         = 'BASE'
# LocationID is an exact string match from here: https://openei.org/datasets/files/961/pub/RESIDENTIAL_LOAD_DATA_E_PLUS_OUTPUT/BASE/
ResLoadAPI['locationID']   = 'USA_CO_Aspen-Pitkin.County-Sardy.Field.724676_TMY3_BASE.csv'

url = '/'.join( str(x) for x in ResLoadAPI.values() )

# Create path to which to save locally
saveDir  = os.path.join( wd, ResLoadAPI['dataCategory'] )
savePath = os.path.join( saveDir, ResLoadAPI['locationID'] )

# Create directory for saving if it doesn't exist
if not os.path.exists(saveDir):
    os.makedirs(saveDir)

DownloadURL( url, savePath )
#Unzip(savePath,destPath)

0

In [71]:
### Rooftop solar profile ###
# Reference, PV watts: https://developer.nrel.gov/docs/solar/pvwatts/v6/#examples
# Example data pull: https://github.com/mpaolino/pypvwatts/blob/master/pypvwatts/pypvwatts.py

# Parse API key
key = GetAPIKey()

# Build query

pvWattsInputs = {
    "api_key": key,
    "system_capacity": "4",
    "lat": "39.40",
    "lon": "-107.21",
    "azimuth": "180",
    "tilt": "40",
    "array_type": "1",
    "module_type": "1",
    "losses": "10",
    "timeframe": "hourly"}

baseURL = 'https://developer.nrel.gov/api/pvwatts/v6.json?'

In [72]:
response = requests.request( 'GET', url = baseURL, params = pvWattsInputs )

In [73]:
result = response.json()

In [74]:
result['outputs'].keys()

dict_keys(['ac_monthly', 'poa_monthly', 'solrad_monthly', 'dc_monthly', 'ac_annual', 'solrad_annual', 'capacity_factor', 'ac', 'poa', 'dn', 'dc', 'df', 'tamb', 'tcell', 'wspd'])

In [98]:
# Extract data of interest and collect in PV_Project
PV_Project  = {}
PV_Project['StationInfo']   = result['station_info']
PV_Project['Generation'] = pd.DataFrame( {'KWh': result['outputs']['ac']} ) # Assume AC configuration

In [99]:
# Add datetime to generation and convert from standard to local time. Assume 2021 is the year in initial version for simplicity.
dateTimeDF = pd.DataFrame( {'DateTime': pd.date_range('2021-01-01', '2022-01-01', freq='1H', closed='left')} )

In [101]:
PV_Project['Generation']['DateTime'] = dateTimeDF['DateTime']

In [102]:
PV_Project['Generation']

,KWh,DateTime
0,0.0,2021-01-01 00:00:00
1,0.0,2021-01-01 01:00:00
2,0.0,2021-01-01 02:00:00
3,0.0,2021-01-01 03:00:00
4,0.0,2021-01-01 04:00:00
...,...,...
8755,0.0,2021-12-31 19:00:00
8756,0.0,2021-12-31 20:00:00
8757,0.0,2021-12-31 21:00:00
8758,0.0,2021-12-31 22:00:00


In [96]:
# Convert to local time

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1472.8,
 2349.033,
 1453.11,
 2528.114,
 1937.986,
 1310.901,
 1166.454,
 1319.346,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 423.045,
 73.324,
 331.966,
 1476.167,
 1267.434,
 208.298,
 473.503,
 1109.901,
 390.938,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 421.368,
 712.131,
 3008.01,
 3235.667,
 3274.357,
 3066.427,
 2622.824,
 1862.678,
 773.619,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 109.01,
 2462.259,
 2927.234,
 3200.828,
 3243.52,
 3039.903,
 2591.538,
 1846.96,
 764.13,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 989.939,
 2320.026,
 2951.012,
 2310.478,
 3324.414,
 1682.64,
 818.515,
 825.11,
 12.727,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 68.574,
 537.132,
 745.914,
 454.043,
